# Air Quality PM2.5 Prediction - Feature Engineering

**Author:** Chukwuahachie Sylvester 
**Date:** December 2, 2024  
**Purpose:** Advanced feature engineering for PM2.5 prediction model

## Objectives
1. Handle missing data with domain knowledge
2. Create domain-specific features (pollutant interactions, wind features)
3. Engineer temporal features (seasonality, time patterns)
4. Build aggregated features (rolling averages, city statistics)
5. Apply dimensionality reduction (PCA on sensor angles)
6. Create pollution clusters
7. Prepare final dataset for modeling

---

In [ ]:
# ============================================
# IMPORT LIBRARIES
# ============================================

# Data manipulation
import pandas as pd
import numpy as np
from datetime import datetime

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Feature engineering
from sklearn.preprocessing import StandardScaler, LabelEncoder, PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.impute import KNNImputer, SimpleImputer

# Model selection
from sklearn.model_selection import train_test_split

# Statistical
from scipy import stats

# Warnings
import warnings
warnings.filterwarnings('ignore')

# Visualization settings
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("All libraries imported successfully!")
print(f"Pandas: {pd.__version__}")
print(f"NumPy: {np.__version__}")

In [ ]:
# ============================================
# LOAD DATA
# ============================================

# Load dataset
DATA_PATH = '../data/raw/air_quality.csv' 

df = pd.read_csv(DATA_PATH)

print(f"Shape: {df.shape}")
print(f"Rows: {df.shape[0]:,}")
print(f"Columns: {df.shape[1]}")

# Display first few rows
print("\n" + "="*60)
print("FIRST 5 ROWS")
print("="*60)
df.head()

In [ ]:
# ============================================
# INITIAL DATA PREPARATION
# ============================================

print("="*60)
print("STEP 1: INITIAL DATA CLEANING")
print("="*60)

# Create a copy for feature engineering
df_fe = df.copy()

# Convert Date to datetime
df_fe['Date'] = pd.to_datetime(df_fe['Date'])
print(f"\n Date converted to datetime")

# Sort by Place and Date (important for time series features)
df_fe = df_fe.sort_values(['Place_ID', 'Date']).reset_index(drop=True)
print(f" Data sorted by Place_ID and Date")

# Display info
print(f"\n Dataset after preparation:")
print(f"Shape: {df_fe.shape}")
print(f"Date range: {df_fe['Date'].min()} to {df_fe['Date'].max()}")
print(f"Number of unique places: {df_fe['Place_ID'].nunique()}")

df_fe.head()

In [ ]:

print("="*60)
print("REMOVING DATA LEAKAGE FEATURES")
print("="*60)

# These features are derived from target - they leak information!
leakage_features = [
    'target_min',
    'target_max', 
    'target_variance',
    'target_count'
]

# Check which exist
existing_leakage = [col for col in leakage_features if col in df_fe.columns]

if existing_leakage:
    print(f"\n Removing {len(existing_leakage)} leakage features:")
    for feature in existing_leakage:
        print(f"   - {feature}")
    
    df_fe = df_fe.drop(columns=existing_leakage)
    print(f"\n Leakage features removed")
    print(f"New shape: {df_fe.shape}")
else:
    print("\n No leakage features found")

# Also remove identifier columns (will add back later)
id_cols = ['Place_ID X Date']
if 'Place_ID X Date' in df_fe.columns:
    df_fe = df_fe.drop(columns=id_cols)
    print(f" Removed identifier column")

print(f"\nFinal shape: {df_fe.shape}")

In [ ]:
# ============================================
# STEP 3: ANALYZE MISSING DATA PATTERNS
# ============================================

print("="*60)
print("MISSING DATA ANALYSIS")
print("="*60)

# Calculate missing percentages
missing_pct = (df_fe.isnull().sum() / len(df_fe) * 100).sort_values(ascending=False)
missing_pct = missing_pct[missing_pct > 0]

print(f"\n Features with missing data: {len(missing_pct)}")

# Categorize by missing percentage
high_missing = missing_pct[missing_pct > 50]
medium_missing = missing_pct[(missing_pct > 10) & (missing_pct <= 50)]
low_missing = missing_pct[missing_pct <= 10]

print(f"\n HIGH missing (>50%): {len(high_missing)} features")
if len(high_missing) > 0:
    print(high_missing.head(10))

print(f"\n MEDIUM missing (10-50%): {len(medium_missing)} features")
if len(medium_missing) > 0:
    print(medium_missing.head(10))

print(f"\n LOW missing (<10%): {len(low_missing)} features")
if len(low_missing) > 0:
    print(low_missing.head(10))

# Visualize
plt.figure(figsize=(14, 6))
missing_pct.head(30).plot(kind='barh', color='coral', edgecolor='black')
plt.xlabel('Missing Percentage (%)')
plt.title('Top 30 Features with Missing Data', fontweight='bold', fontsize=14)
plt.axvline(x=50, color='red', linestyle='--', linewidth=2, label='50% threshold')
plt.axvline(x=10, color='orange', linestyle='--', linewidth=2, label='10% threshold')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# ============================================
# STEP 4: DROP FEATURES WITH >80% MISSING
# ============================================

print("="*60)
print("DROPPING HIGH-MISSING FEATURES")
print("="*60)

# Drop features with >80% missing (CH4 features)
threshold = 80
high_missing_cols = missing_pct[missing_pct > threshold].index.tolist()

print(f"\n Dropping {len(high_missing_cols)} features with >{threshold}% missing:")
for col in high_missing_cols:
    print(f"   - {col}: {missing_pct[col]:.1f}% missing")

df_fe = df_fe.drop(columns=high_missing_cols)

print(f"\n High-missing features dropped")
print(f"New shape: {df_fe.shape}")

In [ ]:
# ============================================
# STEP 5: CREATE TEMPORAL FEATURES
# ============================================

print("="*60)
print("CREATING TEMPORAL FEATURES")
print("="*60)

# Extract temporal components
df_fe['year'] = df_fe['Date'].dt.year
df_fe['month'] = df_fe['Date'].dt.month
df_fe['day'] = df_fe['Date'].dt.day
df_fe['dayofweek'] = df_fe['Date'].dt.dayofweek  # Monday=0, Sunday=6
df_fe['dayofyear'] = df_fe['Date'].dt.dayofyear
df_fe['week'] = df_fe['Date'].dt.isocalendar().week
df_fe['quarter'] = df_fe['Date'].dt.quarter

# Weekend indicator
df_fe['is_weekend'] = (df_fe['dayofweek'] >= 5).astype(int)

# Season (Northern Hemisphere)
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

df_fe['season'] = df_fe['month'].apply(get_season)

# Cyclical encoding for month (preserves cyclical nature)
df_fe['month_sin'] = np.sin(2 * np.pi * df_fe['month'] / 12)
df_fe['month_cos'] = np.cos(2 * np.pi * df_fe['month'] / 12)

# Cyclical encoding for day of week
df_fe['dayofweek_sin'] = np.sin(2 * np.pi * df_fe['dayofweek'] / 7)
df_fe['dayofweek_cos'] = np.cos(2 * np.pi * df_fe['dayofweek'] / 7)

print(f"\n Created {8 + 4} temporal features:")
print(f"   - Basic: year, month, day, dayofweek, dayofyear, week, quarter, season")
print(f"   - Binary: is_weekend")
print(f"   - Cyclical: month_sin/cos, dayofweek_sin/cos")

print(f"\nNew shape: {df_fe.shape}")

# Show sample
print("\n Sample temporal features:")
df_fe[['Date', 'month', 'season', 'is_weekend', 'month_sin', 'month_cos']].head(10)

In [ ]:
# ============================================
# STEP 6: CREATE WIND FEATURES (DOMAIN KNOWLEDGE)
# ============================================

print("="*60)
print("CREATING WIND FEATURES")
print("="*60)

# Wind speed from u and v components
# Formula: wind_speed = sqrt(u² + v²)
if 'u_component_of_wind_10m_above_ground' in df_fe.columns and 'v_component_of_wind_10m_above_ground' in df_fe.columns:
    
    df_fe['wind_speed'] = np.sqrt(
        df_fe['u_component_of_wind_10m_above_ground']**2 + 
        df_fe['v_component_of_wind_10m_above_ground']**2
    )
    
    # Wind direction (in radians, then convert to degrees)
    df_fe['wind_direction'] = np.arctan2(
        df_fe['v_component_of_wind_10m_above_ground'],
        df_fe['u_component_of_wind_10m_above_ground']
    )
    
    # Convert to degrees (0-360)
    df_fe['wind_direction_degrees'] = (df_fe['wind_direction'] * 180 / np.pi) % 360
    
    # Wind direction categories (N, NE, E, SE, S, SW, W, NW)
    def categorize_wind_direction(degrees):
        if pd.isna(degrees):
            return np.nan
        elif degrees < 22.5 or degrees >= 337.5:
            return 'N'
        elif degrees < 67.5:
            return 'NE'
        elif degrees < 112.5:
            return 'E'
        elif degrees < 157.5:
            return 'SE'
        elif degrees < 202.5:
            return 'S'
        elif degrees < 247.5:
            return 'SW'
        elif degrees < 292.5:
            return 'W'
        else:
            return 'NW'
    
    df_fe['wind_direction_category'] = df_fe['wind_direction_degrees'].apply(categorize_wind_direction)
    
    print(f"\n Created wind features:")
    print(f"   - wind_speed (magnitude)")
    print(f"   - wind_direction (radians)")
    print(f"   - wind_direction_degrees (0-360°)")
    print(f"   - wind_direction_category (N, NE, E, etc.)")
    
    # Visualize
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Wind speed distribution
    axes[0].hist(df_fe['wind_speed'].dropna(), bins=50, edgecolor='black', alpha=0.7, color='skyblue')
    axes[0].set_xlabel('Wind Speed (m/s)')
    axes[0].set_ylabel('Frequency')
    axes[0].set_title('Wind Speed Distribution', fontweight='bold')
    axes[0].grid(alpha=0.3)
    
    # Wind direction
    wind_dir_counts = df_fe['wind_direction_category'].value_counts()
    axes[1].bar(wind_dir_counts.index, wind_dir_counts.values, edgecolor='black', alpha=0.7, color='coral')
    axes[1].set_xlabel('Wind Direction')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title('Wind Direction Distribution', fontweight='bold')
    axes[1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
else:
    print("\n Wind component features not found")

print(f"\nNew shape: {df_fe.shape}")

In [ ]:
# ============================================
# STEP 7: CREATE POLLUTANT INTERACTION FEATURES
# ============================================

print("="*60)
print("CREATING POLLUTANT INTERACTION FEATURES")
print("="*60)

# Key pollutants
pollutants = {
    'CO': 'L3_CO_CO_column_number_density',
    'NO2': 'L3_NO2_NO2_column_number_density',
    'SO2': 'L3_SO2_SO2_column_number_density',
    'HCHO': 'L3_HCHO_tropospheric_HCHO_column_number_density'
}

# Check which exist
existing_pollutants = {k: v for k, v in pollutants.items() if v in df_fe.columns}

print(f"\n Found {len(existing_pollutants)} pollutant features:")
for name, col in existing_pollutants.items():
    print(f"   - {name}: {col}")

if len(existing_pollutants) >= 2:
    
    pollutant_cols = list(existing_pollutants.values())
    
    # 1. Total pollutant load (sum of all pollutants)
    df_fe['total_pollutant_load'] = df_fe[pollutant_cols].sum(axis=1)
    
    # 2. Average pollutant concentration
    df_fe['avg_pollutant_concentration'] = df_fe[pollutant_cols].mean(axis=1)
    
    # 3. Pollutant interaction: CO × NO2 (vehicle emissions proxy)
    if 'L3_CO_CO_column_number_density' in df_fe.columns and 'L3_NO2_NO2_column_number_density' in df_fe.columns:
        df_fe['CO_NO2_interaction'] = df_fe['L3_CO_CO_column_number_density'] * df_fe['L3_NO2_NO2_column_number_density']
    
    # 4. Air Quality Index proxy (weighted sum)
    # Weights based on health impact
    if all(col in df_fe.columns for col in pollutant_cols):
        df_fe['AQI_proxy'] = (
            0.4 * df_fe.get('L3_CO_CO_column_number_density', 0) +
            0.3 * df_fe.get('L3_NO2_NO2_column_number_density', 0) +
            0.2 * df_fe.get('L3_SO2_SO2_column_number_density', 0) +
            0.1 * df_fe.get('L3_HCHO_tropospheric_HCHO_column_number_density', 0)
        )
    
    print(f"\nCreated {4} pollutant interaction features:")
    print(f"   - total_pollutant_load")
    print(f"   - avg_pollutant_concentration")
    print(f"   - CO_NO2_interaction")
    print(f"   - AQI_proxy")
    
else:
    print("\n Not enough pollutant features for interactions")

print(f"\nNew shape: {df_fe.shape}")

In [ ]:
# ============================================
# STEP 8: CREATE ATMOSPHERIC STABILITY FEATURES
# ============================================

print("="*60)
print("CREATING ATMOSPHERIC STABILITY FEATURES")
print("="*60)

# Temperature-Humidity Index (measures atmospheric comfort/stability)
if 'temperature_2m_above_ground' in df_fe.columns and 'relative_humidity_2m_above_ground' in df_fe.columns:
    
    # Heat Index (how hot it feels)
    df_fe['heat_index'] = df_fe['temperature_2m_above_ground'] + (
        0.5 * df_fe['relative_humidity_2m_above_ground'] / 100
    )
    
    # Temperature-Pressure interaction (atmospheric stability)
    if 'L3_NO2_tropopause_pressure' in df_fe.columns:
        df_fe['temp_pressure_ratio'] = (
            df_fe['temperature_2m_above_ground'] / 
            (df_fe['L3_NO2_tropopause_pressure'] + 1)  # +1 to avoid division by zero
        )
    
    # Humidity categories
    def categorize_humidity(humidity):
        if pd.isna(humidity):
            return np.nan
        elif humidity < 30:
            return 'Dry'
        elif humidity < 60:
            return 'Moderate'
        else:
            return 'Humid'
    
    df_fe['humidity_category'] = df_fe['relative_humidity_2m_above_ground'].apply(categorize_humidity)
    
    # Temperature-Humidity interaction
    df_fe['temp_humidity_interaction'] = (
        df_fe['temperature_2m_above_ground'] * 
        df_fe['relative_humidity_2m_above_ground']
    )
    
    print(f"\n Created atmospheric stability features:")
    print(f"   - heat_index")
    print(f"   - temp_pressure_ratio")
    print(f"   - humidity_category")
    print(f"   - temp_humidity_interaction")
    
else:
    print("\n Temperature/Humidity features not found")

print(f"\nNew shape: {df_fe.shape}")

In [ ]:
# ============================================
# STEP 9: CREATE LOCATION-BASED AGGREGATED FEATURES
# ============================================

print("="*60)
print("CREATING LOCATION-BASED AGGREGATED FEATURES")
print("="*60)

# Calculate statistics by Place_ID
if 'Place_ID' in df_fe.columns:
    
    # Mean PM2.5 per location (pollution history)
    place_avg_pm25 = df_fe.groupby('Place_ID')['target'].transform('mean')
    df_fe['place_avg_pm25'] = place_avg_pm25
    
    # Standard deviation per location (pollution variability)
    place_std_pm25 = df_fe.groupby('Place_ID')['target'].transform('std')
    df_fe['place_std_pm25'] = place_std_pm25
    
    # Median per location
    place_median_pm25 = df_fe.groupby('Place_ID')['target'].transform('median')
    df_fe['place_median_pm25'] = place_median_pm25
    
    # Min and Max per location
    place_min_pm25 = df_fe.groupby('Place_ID')['target'].transform('min')
    df_fe['place_min_pm25'] = place_min_pm25
    
    place_max_pm25 = df_fe.groupby('Place_ID')['target'].transform('max')
    df_fe['place_max_pm25'] = place_max_pm25
    
    # Number of observations per location
    place_count = df_fe.groupby('Place_ID')['target'].transform('count')
    df_fe['place_observation_count'] = place_count
    
    print(f"\n Created {6} location-based features:")
    print(f"   - place_avg_pm25 (historical average)")
    print(f"   - place_std_pm25 (variability)")
    print(f"   - place_median_pm25")
    print(f"   - place_min_pm25")
    print(f"   - place_max_pm25")
    print(f"   - place_observation_count")
    
    # Visualize top polluted locations
    place_pollution = df_fe.groupby('Place_ID')['target'].mean().sort_values(ascending=False).head(15)
    
    plt.figure(figsize=(12, 6))
    place_pollution.plot(kind='barh', color='crimson', edgecolor='black')
    plt.xlabel('Average PM2.5 (μg/m³)')
    plt.ylabel('Place ID')
    plt.title('Top 15 Most Polluted Locations', fontweight='bold', fontsize=14)
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
    
else:
    print("\n Place_ID column not found")

print(f"\nNew shape: {df_fe.shape}")

In [ ]:
# ============================================
# STEP 10: CREATE ROLLING WINDOW FEATURES
# ============================================

print("="*60)
print("CREATING ROLLING WINDOW FEATURES (TIME SERIES)")
print("="*60)

# Sort by Place_ID and Date (critical for rolling windows)
df_fe = df_fe.sort_values(['Place_ID', 'Date']).reset_index(drop=True)

if 'Place_ID' in df_fe.columns and 'Date' in df_fe.columns:
    
    print("\n Creating rolling averages per location...")
    
    # 3-day rolling average
    df_fe['pm25_rolling_3day'] = df_fe.groupby('Place_ID')['target'].transform(
        lambda x: x.rolling(window=3, min_periods=1).mean()
    )
    
    # 7-day rolling average
    df_fe['pm25_rolling_7day'] = df_fe.groupby('Place_ID')['target'].transform(
        lambda x: x.rolling(window=7, min_periods=1).mean()
    )
    
    # 14-day rolling average
    df_fe['pm25_rolling_14day'] = df_fe.groupby('Place_ID')['target'].transform(
        lambda x: x.rolling(window=14, min_periods=1).mean()
    )
    
    # 30-day rolling average
    df_fe['pm25_rolling_30day'] = df_fe.groupby('Place_ID')['target'].transform(
        lambda x: x.rolling(window=30, min_periods=1).mean()
    )
    
    # Rolling standard deviation (volatility)
    df_fe['pm25_rolling_std_7day'] = df_fe.groupby('Place_ID')['target'].transform(
        lambda x: x.rolling(window=7, min_periods=1).std()
    )
    
    # Lag features (yesterday's pollution)
    df_fe['pm25_lag_1day'] = df_fe.groupby('Place_ID')['target'].shift(1)
    df_fe['pm25_lag_7day'] = df_fe.groupby('Place_ID')['target'].shift(7)
    
    # Difference from yesterday (trend)
    df_fe['pm25_diff_1day'] = df_fe['target'] - df_fe['pm25_lag_1day']
    
    # Difference from 7 days ago (weekly trend)
    df_fe['pm25_diff_7day'] = df_fe['target'] - df_fe['pm25_lag_7day']
    
    print(f"\n Created {9} rolling window features:")
    print(f"   - pm25_rolling_3day, 7day, 14day, 30day")
    print(f"   - pm25_rolling_std_7day")
    print(f"   - pm25_lag_1day, pm25_lag_7day")
    print(f"   - pm25_diff_1day, pm25_diff_7day")
    
    # Visualize for one location
    sample_place = df_fe['Place_ID'].iloc[0]
    sample_data = df_fe[df_fe['Place_ID'] == sample_place].head(60)
    
    plt.figure(figsize=(16, 6))
    plt.plot(sample_data['Date'], sample_data['target'], 'o-', alpha=0.5, label='Actual PM2.5')
    plt.plot(sample_data['Date'], sample_data['pm25_rolling_7day'], linewidth=2, label='7-day Rolling Avg')
    plt.plot(sample_data['Date'], sample_data['pm25_rolling_30day'], linewidth=2, label='30-day Rolling Avg')
    plt.xlabel('Date')
    plt.ylabel('PM2.5 (μg/m³)')
    plt.title(f'Rolling Averages - {sample_place}', fontweight='bold', fontsize=14)
    plt.legend()
    plt.grid(alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
else:
    print("\n Cannot create rolling features without Place_ID and Date")

print(f"\nNew shape: {df_fe.shape}")

In [ ]:
# ============================================
# STEP 11: CREATE POLYNOMIAL FEATURES
# ============================================

print("="*60)
print("CREATING POLYNOMIAL FEATURES FOR TOP PREDICTORS")
print("="*60)

# Top correlated features from EDA
top_features = [
    'L3_CO_CO_column_number_density',
    'L3_HCHO_tropospheric_HCHO_column_number_density',
    'L3_NO2_NO2_column_number_density',
    'temperature_2m_above_ground',
    'relative_humidity_2m_above_ground'
]

# Filter to existing features
top_features = [f for f in top_features if f in df_fe.columns]

print(f"\n Creating polynomial features for {len(top_features)} top predictors:")
for feature in top_features:
    print(f"   - {feature}")

for feature in top_features:
    # Square
    df_fe[f'{feature}_squared'] = df_fe[feature] ** 2
    
    # Square root (for positive values)
    if df_fe[feature].min() >= 0:
        df_fe[f'{feature}_sqrt'] = np.sqrt(df_fe[feature])
    
    # Log transform (for positive values)
    if df_fe[feature].min() > 0:
        df_fe[f'{feature}_log'] = np.log1p(df_fe[feature])

print(f"\n Created polynomial features (squared, sqrt, log)")
print(f"New shape: {df_fe.shape}")

In [ ]:
# ============================================
# STEP 12: CREATE RATIO FEATURES
# ============================================

print("="*60)
print("CREATING RATIO FEATURES")
print("="*60)

# Temperature to humidity ratio (dryness index)
if 'temperature_2m_above_ground' in df_fe.columns and 'relative_humidity_2m_above_ground' in df_fe.columns:
    df_fe['temp_humidity_ratio'] = (
        df_fe['temperature_2m_above_ground'] / 
        (df_fe['relative_humidity_2m_above_ground'] + 1)
    )

# NO2 to CO ratio (traffic vs industrial pollution)
if 'L3_NO2_NO2_column_number_density' in df_fe.columns and 'L3_CO_CO_column_number_density' in df_fe.columns:
    df_fe['NO2_CO_ratio'] = (
        df_fe['L3_NO2_NO2_column_number_density'] / 
        (df_fe['L3_CO_CO_column_number_density'] + 1e-6)
    )

# Pollutant concentration per wind speed (dispersion efficiency)
if 'total_pollutant_load' in df_fe.columns and 'wind_speed' in df_fe.columns:
    df_fe['pollutant_per_windspeed'] = (
        df_fe['total_pollutant_load'] / 
        (df_fe['wind_speed'] + 0.1)  # Avoid division by zero
    )

print(f"\n Created ratio features:")
print(f"   - temp_humidity_ratio")
print(f"   - NO2_CO_ratio")
print(f"   - pollutant_per_windspeed")

print(f"\nNew shape: {df_fe.shape}")

In [ ]:
# ============================================
# STEP 13: APPLY PCA FOR DIMENSIONALITY REDUCTION
# ============================================

print("="*60)
print("APPLYING PCA FOR DIMENSIONALITY REDUCTION")
print("="*60)

# Find all sensor angle features (highly correlated, low predictive power)
angle_features = [col for col in df_fe.columns if 'angle' in col.lower() and col in df_fe.select_dtypes(include=[np.number]).columns]

print(f"\n Found {len(angle_features)} sensor angle features:")
for feature in angle_features[:10]:
    print(f"   - {feature}")
if len(angle_features) > 10:
    print(f"   ... and {len(angle_features) - 10} more")

if len(angle_features) >= 5:
    
    print(f"\n Reducing {len(angle_features)} angle features to 5 principal components...")
    
    # Prepare data for PCA (handle missing values)
    angle_data = df_fe[angle_features].copy()
    
    # Impute missing values with median
    imputer = SimpleImputer(strategy='median')
    angle_data_imputed = imputer.fit_transform(angle_data)
    
    # Standardize
    scaler = StandardScaler()
    angle_data_scaled = scaler.fit_transform(angle_data_imputed)
    
    # Apply PCA
    n_components = min(5, len(angle_features))
    pca = PCA(n_components=n_components, random_state=42)
    angle_pca = pca.fit_transform(angle_data_scaled)
    
    # Add PCA components to dataframe
    for i in range(n_components):
        df_fe[f'angle_pca_{i+1}'] = angle_pca[:, i]
    
    # Calculate explained variance
    explained_var = pca.explained_variance_ratio_
    cumulative_var = np.cumsum(explained_var)
    
    print(f"\n PCA completed!")
    print(f"   - Created {n_components} PCA components")
    print(f"   - Explained variance: {explained_var}")
    print(f"   - Cumulative variance: {cumulative_var}")
    
    # Visualize
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.bar(range(1, n_components+1), explained_var, edgecolor='black', alpha=0.7, color='steelblue')
    plt.xlabel('Principal Component')
    plt.ylabel('Explained Variance Ratio')
    plt.title('PCA - Explained Variance per Component', fontweight='bold')
    plt.grid(alpha=0.3)
    
    plt.subplot(1, 2, 2)
    plt.plot(range(1, n_components+1), cumulative_var, 'o-', linewidth=2, markersize=8, color='coral')
    plt.xlabel('Number of Components')
    plt.ylabel('Cumulative Explained Variance')
    plt.title('PCA - Cumulative Explained Variance', fontweight='bold')
    plt.grid(alpha=0.3)
    plt.axhline(y=0.8, color='red', linestyle='--', label='80% threshold')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    # Drop original angle features (optional - keep for now)
    # df_fe = df_fe.drop(columns=angle_features)
    # print(f"\n Dropped {len(angle_features)} original angle features")
    
else:
    print(f"\n Not enough angle features for PCA (found {len(angle_features)}, need at least 5)")

print(f"\nNew shape: {df_fe.shape}")

In [ ]:
# ============================================
# STEP 14: CREATE POLLUTION LEVEL CLUSTERS
# ============================================

print("="*60)
print("CREATING POLLUTION LEVEL CLUSTERS")
print("="*60)

# Features for clustering
cluster_features = [
    'target',
    'L3_CO_CO_column_number_density',
    'L3_NO2_NO2_column_number_density',
    'temperature_2m_above_ground',
    'relative_humidity_2m_above_ground',
    'wind_speed'
]

# Filter to existing features
cluster_features = [f for f in cluster_features if f in df_fe.columns]

print(f"\n Using {len(cluster_features)} features for clustering:")
for feature in cluster_features:
    print(f"   - {feature}")

if len(cluster_features) >= 3:
    
    # Prepare data
    cluster_data = df_fe[cluster_features].copy()
    
    # Handle missing values
    imputer = SimpleImputer(strategy='median')
    cluster_data_imputed = imputer.fit_transform(cluster_data)
    
    # Standardize
    scaler = StandardScaler()
    cluster_data_scaled = scaler.fit_transform(cluster_data_imputed)
    
    # Apply K-Means clustering
    n_clusters = 4  # Low, Medium, High, Very High pollution zones
    
    print(f"\n Applying K-Means clustering with {n_clusters} clusters...")
    
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    df_fe['pollution_cluster'] = kmeans.fit_predict(cluster_data_scaled)
    
    print(f"\n Clustering completed!")
    
    # Analyze clusters
    print(f"\n Cluster Analysis:")
    cluster_stats = df_fe.groupby('pollution_cluster')['target'].agg(['mean', 'median', 'std', 'count'])
    cluster_stats = cluster_stats.sort_values('mean')
    cluster_stats.index = [f'Cluster {i}' for i in cluster_stats.index]
    print(cluster_stats)
    
    # Rename clusters based on pollution level
    cluster_mapping = dict(zip(
        cluster_stats.sort_values('mean').index,
        ['Low', 'Medium', 'High', 'Very High']
    ))
    
    # Visualize clusters
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Cluster distribution
    cluster_counts = df_fe['pollution_cluster'].value_counts().sort_index()
    axes[0].bar(cluster_counts.index, cluster_counts.values, edgecolor='black', alpha=0.7, color='teal')
    axes[0].set_xlabel('Cluster')
    axes[0].set_ylabel('Count')
    axes[0].set_title('Pollution Cluster Distribution', fontweight='bold')
    axes[0].grid(alpha=0.3)
    
    # Box plot of PM2.5 by cluster
    df_fe.boxplot(column='target', by='pollution_cluster', ax=axes[1], patch_artist=True)
    axes[1].set_xlabel('Pollution Cluster')
    axes[1].set_ylabel('PM2.5 (μg/m³)')
    axes[1].set_title('PM2.5 Distribution by Cluster', fontweight='bold')
    axes[1].get_figure().suptitle('')  # Remove auto title
    axes[1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
else:
    print(f"\n Not enough features for clustering")

print(f"\nNew shape: {df_fe.shape}")

In [ ]:
# ============================================
# STEP 16: TRANSFORM TARGET VARIABLE
# ============================================

print("="*60)
print("TRANSFORMING TARGET VARIABLE")
print("="*60)

# Analyze target distribution before transformation
print("\n Target Variable (PM2.5) - Before Transformation:")
print(f"Mean: {df_fe['target'].mean():.2f}")
print(f"Median: {df_fe['target'].median():.2f}")
print(f"Std: {df_fe['target'].std():.2f}")
print(f"Skewness: {df_fe['target'].skew():.3f}")
print(f"Kurtosis: {df_fe['target'].kurtosis():.3f}")
print(f"Min: {df_fe['target'].min():.2f}")
print(f"Max: {df_fe['target'].max():.2f}")

# Visualize before transformation
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Original distribution
axes[0, 0].hist(df_fe['target'], bins=50, edgecolor='black', alpha=0.7, color='coral')
axes[0, 0].set_xlabel('PM2.5 (μg/m³)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Original Target Distribution', fontweight='bold')
axes[0, 0].axvline(df_fe['target'].mean(), color='red', linestyle='--', linewidth=2, label='Mean')
axes[0, 0].axvline(df_fe['target'].median(), color='blue', linestyle='--', linewidth=2, label='Median')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# Original Q-Q plot
stats.probplot(df_fe['target'], dist="norm", plot=axes[0, 1])
axes[0, 1].set_title('Q-Q Plot - Original', fontweight='bold')
axes[0, 1].grid(alpha=0.3)

# Apply log transformation
# Using log1p to handle zeros: log1p(x) = log(1 + x)
df_fe['target_log'] = np.log1p(df_fe['target'])

print(f"\n Log transformation applied: target_log = log(1 + target)")

# Analyze after transformation
print("\n Target Variable - After Log Transformation:")
print(f"Mean: {df_fe['target_log'].mean():.2f}")
print(f"Median: {df_fe['target_log'].median():.2f}")
print(f"Std: {df_fe['target_log'].std():.2f}")
print(f"Skewness: {df_fe['target_log'].skew():.3f}")
print(f"Kurtosis: {df_fe['target_log'].kurtosis():.3f}")

# Log-transformed distribution
axes[1, 0].hist(df_fe['target_log'], bins=50, edgecolor='black', alpha=0.7, color='lightgreen')
axes[1, 0].set_xlabel('log(PM2.5 + 1)')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].set_title('Log-Transformed Target Distribution', fontweight='bold')
axes[1, 0].axvline(df_fe['target_log'].mean(), color='red', linestyle='--', linewidth=2, label='Mean')
axes[1, 0].axvline(df_fe['target_log'].median(), color='blue', linestyle='--', linewidth=2, label='Median')
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

# Log-transformed Q-Q plot
stats.probplot(df_fe['target_log'], dist="norm", plot=axes[1, 1])
axes[1, 1].set_title('Q-Q Plot - Log-Transformed', fontweight='bold')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Compare skewness reduction
skew_reduction = ((df_fe['target'].skew() - df_fe['target_log'].skew()) / df_fe['target'].skew()) * 100
print(f"\n Skewness reduced by {skew_reduction:.1f}%")
print(f"   Original skewness: {df_fe['target'].skew():.3f}")
print(f"   Log skewness: {df_fe['target_log'].skew():.3f}")

if abs(df_fe['target_log'].skew()) < 0.5:
    print(f"\n Distribution is now approximately normal!")
else:
    print(f"\n Distribution still skewed but improved")

print(f"\nNew shape: {df_fe.shape}")

In [ ]:
# ============================================
# STEP 17: HANDLE OUTLIERS
# ============================================

print("="*60)
print("HANDLING OUTLIERS")
print("="*60)

# Function to detect outliers using IQR method
def detect_outliers_iqr(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return lower_bound, upper_bound

# Function to cap outliers
def cap_outliers(series, lower_bound, upper_bound):
    return series.clip(lower=lower_bound, upper=upper_bound)

# Get numerical features (excluding target)
numerical_features = df_fe.select_dtypes(include=[np.number]).columns.tolist()
features_to_exclude = ['target', 'target_log', 'Date', 'year', 'month', 'day', 'dayofweek']
numerical_features = [f for f in numerical_features if f not in features_to_exclude]

print(f"\n Analyzing outliers in {len(numerical_features)} numerical features...")

# Track outlier statistics
outlier_stats = []

for feature in numerical_features:
    # Count outliers before
    lower, upper = detect_outliers_iqr(df_fe[feature])
    outliers_before = ((df_fe[feature] < lower) | (df_fe[feature] > upper)).sum()
    outliers_pct_before = (outliers_before / len(df_fe)) * 100
    
    if outliers_before > 0:
        outlier_stats.append({
            'Feature': feature,
            'Outliers_Before': outliers_before,
            'Outliers_Pct': outliers_pct_before,
            'Lower_Bound': lower,
            'Upper_Bound': upper
        })

# Sort by percentage
outlier_df = pd.DataFrame(outlier_stats).sort_values('Outliers_Pct', ascending=False)

print(f"\n Top 15 Features with Most Outliers:")
print(outlier_df.head(15)[['Feature', 'Outliers_Before', 'Outliers_Pct']])

# Visualize top outlier features
top_outlier_features = outlier_df.head(6)['Feature'].tolist()

if len(top_outlier_features) > 0:
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    axes = axes.flatten()
    
    for idx, feature in enumerate(top_outlier_features):
        # Before capping
        axes[idx].boxplot(df_fe[feature].dropna(), vert=True, patch_artist=True)
        axes[idx].set_ylabel(feature[:30] + '...' if len(feature) > 30 else feature)
        axes[idx].set_title(f'Outliers: {feature[:25]}...', fontweight='bold', fontsize=10)
        axes[idx].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Apply outlier capping strategy
print(f"\n Capping outliers using IQR method (1.5 * IQR)...")

outliers_capped_count = 0

for feature in numerical_features:
    lower, upper = detect_outliers_iqr(df_fe[feature])
    outliers_before = ((df_fe[feature] < lower) | (df_fe[feature] > upper)).sum()
    
    if outliers_before > 0:
        # Cap outliers
        df_fe[feature] = cap_outliers(df_fe[feature], lower, upper)
        
        # Count remaining outliers
        outliers_after = ((df_fe[feature] < lower) | (df_fe[feature] > upper)).sum()
        outliers_capped_count += (outliers_before - outliers_after)

print(f"\n Outlier handling completed!")
print(f"   Total outliers capped: {outliers_capped_count:,}")

# Visualize after capping
if len(top_outlier_features) > 0:
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    axes = axes.flatten()
    
    for idx, feature in enumerate(top_outlier_features):
        # After capping
        axes[idx].boxplot(df_fe[feature].dropna(), vert=True, patch_artist=True, 
                         boxprops=dict(facecolor='lightgreen'))
        axes[idx].set_ylabel(feature[:30] + '...' if len(feature) > 30 else feature)
        axes[idx].set_title(f'After Capping: {feature[:25]}...', fontweight='bold', fontsize=10)
        axes[idx].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()

print(f"\nNew shape: {df_fe.shape}")

In [ ]:
# ============================================
# STEP 18: ENCODE CATEGORICAL FEATURES
# ============================================

print("="*60)
print("ENCODING CATEGORICAL FEATURES")
print("="*60)

# Identify categorical features
categorical_features = df_fe.select_dtypes(include=['object']).columns.tolist()

print(f"\n Found {len(categorical_features)} categorical features:")
for feature in categorical_features:
    unique_count = df_fe[feature].nunique()
    print(f"   - {feature}: {unique_count} unique values")

# Encode categorical features
if len(categorical_features) > 0:
    
    for feature in categorical_features:
        unique_count = df_fe[feature].nunique()
        
        # Strategy 1: Label Encoding for ordinal or low cardinality
        if unique_count <= 10:
            print(f"\n Label Encoding: {feature} ({unique_count} categories)")
            le = LabelEncoder()
            df_fe[f'{feature}_encoded'] = le.fit_transform(df_fe[feature].astype(str))
            
            # Show mapping
            mapping = dict(zip(le.classes_, le.transform(le.classes_)))
            print(f"   Mapping: {mapping}")
        
        # Strategy 2: One-Hot Encoding for nominal with moderate cardinality
        elif 10 < unique_count <= 50:
            print(f"\n One-Hot Encoding: {feature} ({unique_count} categories)")
            # Get top 10 most frequent categories
            top_categories = df_fe[feature].value_counts().head(10).index.tolist()
            
            for category in top_categories:
                df_fe[f'{feature}_{category}'] = (df_fe[feature] == category).astype(int)
            
            print(f"   Created {len(top_categories)} binary features")
        
        # Strategy 3: Target encoding for high cardinality
        else:
            print(f"\n Target Encoding: {feature} ({unique_count} categories - high cardinality)")
            # Mean target value per category
            target_means = df_fe.groupby(feature)['target'].mean()
            df_fe[f'{feature}_target_encoded'] = df_fe[feature].map(target_means)
            
            print(f"   Mapped to target mean values")
    
    print(f"\n Categorical encoding completed!")
    
    # Optional: Drop original categorical columns
    # df_fe = df_fe.drop(columns=categorical_features)
    # print(f"   Dropped {len(categorical_features)} original categorical features")
    
else:
    print("\n No categorical features to encode")

print(f"\nNew shape: {df_fe.shape}")

In [ ]:
# ============================================
# STEP 19: COMPREHENSIVE FEATURE SELECTION
# ============================================

print("="*60)
print("COMPREHENSIVE FEATURE SELECTION")
print("="*60)

# Prepare data for feature selection
# Get all numerical features (exclude IDs, dates, and original target)
features_to_exclude = ['target', 'Date', 'Place_ID', 'Place_ID X Date']
all_features = [col for col in df_fe.columns if col not in features_to_exclude]
all_features = [col for col in all_features if df_fe[col].dtype in ['int64', 'float64']]

print(f"\n Starting with {len(all_features)} potential features")

# Use log-transformed target for modeling
X = df_fe[all_features].drop('target_log',axis = 1).copy()
y = df_fe['target_log'].copy()

# Handle any remaining NaN values
X = X.fillna(X.median())

print(f"\n Feature matrix:")
print(f"   Shape: {X.shape}")
print(f"   Features: {len(all_features)}")
print(f"   Samples: {len(X)}")

# Split data for feature selection
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"\n Data split:")
print(f"   Training: {X_train.shape}")
print(f"   Testing: {X_test.shape}")

In [ ]:
# ============================================
# METHOD 1: VARIANCE THRESHOLD
# ============================================

print("\n" + "="*60)
print("METHOD 1: VARIANCE THRESHOLD")
print("="*60)

from sklearn.feature_selection import VarianceThreshold

# Remove low variance features (almost constant)
variance_threshold = 0.01  # Features with variance < 0.01 are removed

print(f"\n Removing features with variance < {variance_threshold}...")

# Standardize first (variance threshold works better on scaled data)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Apply variance threshold
var_selector = VarianceThreshold(threshold=variance_threshold)
X_train_var = var_selector.fit_transform(X_train_scaled)

# Get selected features
selected_features_var = X_train.columns[var_selector.get_support()].tolist()

print(f"\n Variance Threshold completed!")
print(f"   Features before: {X_train.shape[1]}")
print(f"   Features after: {len(selected_features_var)}")
print(f"   Features removed: {X_train.shape[1] - len(selected_features_var)}")

# Show removed features
removed_features_var = [f for f in X_train.columns if f not in selected_features_var]
if len(removed_features_var) > 0:
    print(f"\n Removed low-variance features ({len(removed_features_var)}):")
    for feature in removed_features_var[:10]:
        print(f"   - {feature}")
    if len(removed_features_var) > 10:
        print(f"   ... and {len(removed_features_var) - 10} more")

In [ ]:
# ============================================
# METHOD 2: CORRELATION FILTER
# ============================================

print("\n" + "="*60)
print("METHOD 2: CORRELATION FILTER (Remove Multicollinearity)")
print("="*60)

# Work with variance-filtered features
X_train_corr = X_train[selected_features_var].copy()

# Calculate correlation matrix
corr_matrix = X_train_corr.corr().abs()

print(f"\n Finding highly correlated feature pairs...")

# Find pairs with correlation > 0.95
high_corr_threshold = 0.95
upper_triangle = corr_matrix.where(
    np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
)

# Find features with correlation > threshold
to_drop_corr = [column for column in upper_triangle.columns 
                if any(upper_triangle[column] > high_corr_threshold)]

print(f"\n Correlation filter completed!")
print(f"   Features before: {len(selected_features_var)}")
print(f"   Highly correlated features found: {len(to_drop_corr)}")

if len(to_drop_corr) > 0:
    print(f"\n🗑️ Removing highly correlated features:")
    for feature in to_drop_corr[:15]:
        print(f"   - {feature}")
    if len(to_drop_corr) > 15:
        print(f"   ... and {len(to_drop_corr) - 15} more")

# Remove correlated features
selected_features_corr = [f for f in selected_features_var if f not in to_drop_corr]
print(f"   Features after: {len(selected_features_corr)}")

# Visualize correlation matrix (top 30 features)
top_features_viz = selected_features_corr[:30]
plt.figure(figsize=(16, 14))
sns.heatmap(
    X_train[top_features_viz].corr(),
    annot=False,
    cmap='coolwarm',
    center=0,
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.8}
)
plt.title('Feature Correlation Matrix (Top 30 Features)', fontweight='bold', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# ============================================
# METHOD 3: SELECTKBEST (F-STATISTIC & MUTUAL INFORMATION)
# ============================================

print("\n" + "="*60)
print("METHOD 3: SELECTKBEST - STATISTICAL TESTS")
print("="*60)

from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression

# Work with correlation-filtered features
X_train_kbest = X_train[selected_features_corr].copy()
X_test_kbest = X_test[selected_features_corr].copy()

# Method 3A: F-statistic (ANOVA)
print(f"\n Method 3A: F-statistic (ANOVA F-test)...")

k_features = min(100, len(selected_features_corr))  # Select top 100 or all if less

selector_f = SelectKBest(score_func=f_regression, k=k_features)
selector_f.fit(X_train_kbest, y_train)

# Get scores
f_scores = pd.DataFrame({
    'Feature': X_train_kbest.columns,
    'F_Score': selector_f.scores_
}).sort_values('F_Score', ascending=False)

print(f"\n F-statistic scoring completed!")
print(f"\n Top 15 Features by F-Score:")
print(f_scores.head(15))

# Method 3B: Mutual Information
print(f"\n Method 3B: Mutual Information...")

selector_mi = SelectKBest(score_func=mutual_info_regression, k=k_features)
selector_mi.fit(X_train_kbest, y_train)

# Get scores
mi_scores = pd.DataFrame({
    'Feature': X_train_kbest.columns,
    'MI_Score': selector_mi.scores_
}).sort_values('MI_Score', ascending=False)

print(f"\n Mutual Information scoring completed!")
print(f"\n Top 15 Features by MI Score:")
print(mi_scores.head(15))

# Combine scores (normalized)
from sklearn.preprocessing import MinMaxScaler

scaler_scores = MinMaxScaler()

f_scores_norm = scaler_scores.fit_transform(f_scores[['F_Score']])
mi_scores_norm = scaler_scores.fit_transform(mi_scores[['MI_Score']])

combined_scores = pd.DataFrame({
    'Feature': f_scores['Feature'],
    'F_Score_Norm': f_scores_norm.flatten(),
    'MI_Score_Norm': mi_scores_norm.flatten()
})

combined_scores['Combined_Score'] = (
    0.5 * combined_scores['F_Score_Norm'] + 
    0.5 * combined_scores['MI_Score_Norm']
)

combined_scores = combined_scores.sort_values('Combined_Score', ascending=False)

print(f"\n Top 20 Features by Combined Statistical Score:")
print(combined_scores.head(20)[['Feature', 'Combined_Score']])

# Visualize top features
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# F-scores
top_f = f_scores.head(20)
axes[0].barh(range(len(top_f)), top_f['F_Score'], color='steelblue', edgecolor='black')
axes[0].set_yticks(range(len(top_f)))
axes[0].set_yticklabels([f[:30] + '...' if len(f) > 30 else f for f in top_f['Feature']])
axes[0].set_xlabel('F-Score')
axes[0].set_title('Top 20 Features - F-Statistic', fontweight='bold')
axes[0].invert_yaxis()
axes[0].grid(alpha=0.3)

# MI scores
top_mi = mi_scores.head(20)
axes[1].barh(range(len(top_mi)), top_mi['MI_Score'], color='coral', edgecolor='black')
axes[1].set_yticks(range(len(top_mi)))
axes[1].set_yticklabels([f[:30] + '...' if len(f) > 30 else f for f in top_mi['Feature']])
axes[1].set_xlabel('Mutual Information Score')
axes[1].set_title('Top 20 Features - Mutual Information', fontweight='bold')
axes[1].invert_yaxis()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Select top K features from combined score
k_best = min(80, len(combined_scores))
selected_features_statistical = combined_scores.head(k_best)['Feature'].tolist()

print(f"\nSelected {len(selected_features_statistical)} features from statistical methods")

In [ ]:
# ============================================
# METHOD 4: TREE-BASED FEATURE IMPORTANCE
# ============================================

print("\n" + "="*60)
print("METHOD 4: TREE-BASED FEATURE IMPORTANCE")
print("="*60)

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor

# Work with correlation-filtered features
X_train_tree = X_train[selected_features_corr].copy()
X_test_tree = X_test[selected_features_corr].copy()

# Method 4A: Random Forest
print(f"\n Method 4A: Random Forest Feature Importance...")

rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_train_tree, y_train)

rf_importance = pd.DataFrame({
    'Feature': X_train_tree.columns,
    'RF_Importance': rf_model.feature_importances_
}).sort_values('RF_Importance', ascending=False)

print(f" Random Forest completed!")
print(f"\n Top 15 Features by RF Importance:")
print(rf_importance.head(15))

# Method 4B: Gradient Boosting
print(f"\n Method 4B: Gradient Boosting Feature Importance...")

gb_model = GradientBoostingRegressor(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    random_state=42
)

gb_model.fit(X_train_tree, y_train)

gb_importance = pd.DataFrame({
    'Feature': X_train_tree.columns,
    'GB_Importance': gb_model.feature_importances_
}).sort_values('GB_Importance', ascending=False)

print(f" Gradient Boosting completed!")
print(f"\n Top 15 Features by GB Importance:")
print(gb_importance.head(15))

# Method 4C: XGBoost
print(f"\n Method 4C: XGBoost Feature Importance...")

xgb_model = XGBRegressor(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    random_state=42,
    n_jobs=-1
)

xgb_model.fit(X_train_tree, y_train)

xgb_importance = pd.DataFrame({
    'Feature': X_train_tree.columns,
    'XGB_Importance': xgb_model.feature_importances_
}).sort_values('XGB_Importance', ascending=False)

print(f" XGBoost completed!")
print(f"\n Top 15 Features by XGB Importance:")
print(xgb_importance.head(15))

# Combine tree-based importances
tree_importance_combined = rf_importance.merge(gb_importance, on='Feature')
tree_importance_combined = tree_importance_combined.merge(xgb_importance, on='Feature')

# Normalize and average
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

tree_importance_combined['RF_Norm'] = scaler.fit_transform(
    tree_importance_combined[['RF_Importance']]
)
tree_importance_combined['GB_Norm'] = scaler.fit_transform(
    tree_importance_combined[['GB_Importance']]
)
tree_importance_combined['XGB_Norm'] = scaler.fit_transform(
    tree_importance_combined[['XGB_Importance']]
)

# Average importance
tree_importance_combined['Avg_Tree_Importance'] = (
    tree_importance_combined['RF_Norm'] +
    tree_importance_combined['GB_Norm'] +
    tree_importance_combined['XGB_Norm']
) / 3

tree_importance_combined = tree_importance_combined.sort_values(
    'Avg_Tree_Importance', ascending=False
)

print(f"\n Top 20 Features by Average Tree Importance:")
print(tree_importance_combined.head(20)[['Feature', 'Avg_Tree_Importance']])

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# Random Forest
top_rf = rf_importance.head(20)
axes[0].barh(range(len(top_rf)), top_rf['RF_Importance'], color='forestgreen', edgecolor='black')
axes[0].set_yticks(range(len(top_rf)))
axes[0].set_yticklabels([f[:25] + '...' if len(f) > 25 else f for f in top_rf['Feature']], fontsize=8)
axes[0].set_xlabel('Importance')
axes[0].set_title('Random Forest - Top 20', fontweight='bold')
axes[0].invert_yaxis()
axes[0].grid(alpha=0.3)

# Gradient Boosting
top_gb = gb_importance.head(20)
axes[1].barh(range(len(top_gb)), top_gb['GB_Importance'], color='orange', edgecolor='black')
axes[1].set_yticks(range(len(top_gb)))
axes[1].set_yticklabels([f[:25] + '...' if len(f) > 25 else f for f in top_gb['Feature']], fontsize=8)
axes[1].set_xlabel('Importance')
axes[1].set_title('Gradient Boosting - Top 20', fontweight='bold')
axes[1].invert_yaxis()
axes[1].grid(alpha=0.3)

# XGBoost
top_xgb = xgb_importance.head(20)
axes[2].barh(range(len(top_xgb)), top_xgb['XGB_Importance'], color='purple', edgecolor='black')
axes[2].set_yticks(range(len(top_xgb)))
axes[2].set_yticklabels([f[:25] + '...' if len(f) > 25 else f for f in top_xgb['Feature']], fontsize=8)
axes[2].set_xlabel('Importance')
axes[2].set_title('XGBoost - Top 20', fontweight='bold')
axes[2].invert_yaxis()
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Select top features from tree models
k_tree = min(70, len(tree_importance_combined))
selected_features_tree = tree_importance_combined.head(k_tree)['Feature'].tolist()

print(f"\n Selected {len(selected_features_tree)} features from tree-based methods")

In [ ]:
# ============================================
# STEP 20: COMBINE ALL FEATURE SELECTION METHODS
# ============================================

print("="*60)
print("COMBINING ALL FEATURE SELECTION METHODS")
print("="*60)

print(f"\n Summary of Feature Selection Methods:")
print(f"   1. Variance Threshold: {len(selected_features_var)} features")
print(f"   2. Correlation Filter: {len(selected_features_corr)} features")
print(f"   3. Statistical (F-stat + MI): {len(selected_features_statistical)} features")
print(f"   4. Tree-Based (RF + GB + XGB): {len(selected_features_tree)} features")

# Create a comprehensive scoring system
print(f"\n Creating comprehensive feature scoring system...")

# Initialize score dataframe
all_features_scores = pd.DataFrame({'Feature': selected_features_corr})

# Add scores from each method

# 1. Statistical scores (normalized)
statistical_scores_dict = dict(zip(
    combined_scores['Feature'],
    combined_scores['Combined_Score']
))
all_features_scores['Statistical_Score'] = all_features_scores['Feature'].map(
    statistical_scores_dict
).fillna(0)

# 2. Tree-based scores (normalized)
tree_scores_dict = dict(zip(
    tree_importance_combined['Feature'],
    tree_importance_combined['Avg_Tree_Importance']
))
all_features_scores['Tree_Score'] = all_features_scores['Feature'].map(
    tree_scores_dict
).fillna(0)

# 3. Selection frequency (how many methods selected this feature)
all_features_scores['Selection_Count'] = all_features_scores['Feature'].apply(
    lambda x: sum([
        x in selected_features_var,
        x in selected_features_corr,
        x in selected_features_statistical,
        x in selected_features_tree
    ])
)

# 4. Calculate final composite score
# Weight: 40% tree-based, 40% statistical, 20% selection frequency
all_features_scores['Final_Score'] = (
    0.40 * all_features_scores['Tree_Score'] +
    0.40 * all_features_scores['Statistical_Score'] +
    0.20 * (all_features_scores['Selection_Count'] / 4)
)

# Sort by final score
all_features_scores = all_features_scores.sort_values('Final_Score', ascending=False)

print(f"\n Feature scoring completed!")
print(f"\n Top 30 Features by Final Composite Score:")
print(all_features_scores.head(30))

# Visualize top features
top_features_final = all_features_scores.head(30)

fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# Bar plot of final scores
axes[0].barh(range(len(top_features_final)), top_features_final['Final_Score'], 
             color='teal', edgecolor='black', alpha=0.7)
axes[0].set_yticks(range(len(top_features_final)))
axes[0].set_yticklabels([f[:35] + '...' if len(f) > 35 else f 
                         for f in top_features_final['Feature']], fontsize=9)
axes[0].set_xlabel('Final Composite Score', fontsize=11)
axes[0].set_title('Top 30 Features - Final Score', fontweight='bold', fontsize=13)
axes[0].invert_yaxis()
axes[0].grid(alpha=0.3, axis='x')

# Stacked scores breakdown
top_20 = all_features_scores.head(20)
x_pos = np.arange(len(top_20))

axes[1].barh(x_pos, top_20['Tree_Score'], 
            label='Tree-Based (40%)', color='forestgreen', alpha=0.8)
axes[1].barh(x_pos, top_20['Statistical_Score'], 
            left=top_20['Tree_Score'],
            label='Statistical (40%)', color='steelblue', alpha=0.8)
axes[1].barh(x_pos, top_20['Selection_Count']/4 * 0.5,
            left=top_20['Tree_Score'] + top_20['Statistical_Score'],
            label='Selection Freq (20%)', color='coral', alpha=0.8)

axes[1].set_yticks(x_pos)
axes[1].set_yticklabels([f[:30] + '...' if len(f) > 30 else f 
                         for f in top_20['Feature']], fontsize=8)
axes[1].set_xlabel('Score Contribution', fontsize=11)
axes[1].set_title('Top 20 Features - Score Breakdown', fontweight='bold', fontsize=13)
axes[1].invert_yaxis()
axes[1].legend(loc='lower right')
axes[1].grid(alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

In [ ]:
# ============================================
# STEP 21: RECURSIVE FEATURE ELIMINATION (RFECV)
# ============================================

print("\n" + "="*60)
print("RECURSIVE FEATURE ELIMINATION WITH CROSS-VALIDATION (RFECV)")
print("="*60)

from sklearn.feature_selection import RFECV
from sklearn.linear_model import Ridge

# Use top 100 features from composite scoring
top_100_features = all_features_scores.head(100)['Feature'].tolist()

X_train_rfecv = X_train[top_100_features].copy()
X_test_rfecv = X_test[top_100_features].copy()

print(f"\n Running RFECV on top 100 features...")
print(f"   This may take a few minutes...")

# Use Ridge regression for RFECV (faster than tree models)
estimator = Ridge(alpha=1.0, random_state=42)

# RFECV with 5-fold cross-validation
rfecv = RFECV(
    estimator=estimator,
    step=5,  # Remove 5 features at a time
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=0
)

rfecv.fit(X_train_rfecv, y_train)

print(f"\n RFECV completed!")
print(f"   Optimal number of features: {rfecv.n_features_}")
print(f"   Features eliminated: {len(top_100_features) - rfecv.n_features_}")

# Get selected features
selected_features_rfecv = X_train_rfecv.columns[rfecv.support_].tolist()

print(f"\n RFECV Selected Features ({len(selected_features_rfecv)}):")
for i, feature in enumerate(selected_features_rfecv[:20], 1):
    print(f"   {i}. {feature}")
if len(selected_features_rfecv) > 20:
    print(f"   ... and {len(selected_features_rfecv) - 20} more")

# Plot cross-validation scores
plt.figure(figsize=(12, 6))
plt.plot(range(1, len(rfecv.cv_results_['mean_test_score']) + 1),
         -rfecv.cv_results_['mean_test_score'],
         marker='o', linewidth=2, markersize=6, color='darkblue')
plt.axvline(x=rfecv.n_features_, color='red', linestyle='--', linewidth=2,
            label=f'Optimal: {rfecv.n_features_} features')
plt.xlabel('Number of Features Selected', fontsize=12)
plt.ylabel('Cross-Validation MSE', fontsize=12)
plt.title('RFECV - Feature Selection Performance', fontweight='bold', fontsize=14)
plt.legend(fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# ============================================
# STEP 22: FINAL FEATURE SET SELECTION (ENSEMBLE VOTING)
# ============================================

print("\n" + "="*60)
print("FINAL FEATURE SET SELECTION - ENSEMBLE VOTING")
print("="*60)

# Create voting system across all methods
print(f"\n Applying ensemble voting across all selection methods...")

# Get top N from each method
top_k = 60  # Take top 60 from each method

top_statistical = combined_scores.head(top_k)['Feature'].tolist()
top_tree = tree_importance_combined.head(top_k)['Feature'].tolist()
top_rfecv = selected_features_rfecv

# Count votes for each feature
feature_votes = {}

for feature in selected_features_corr:
    votes = 0
    
    # Vote from statistical methods
    if feature in top_statistical:
        rank = top_statistical.index(feature) + 1
        votes += (top_k - rank + 1) / top_k  # Higher rank = more votes
    
    # Vote from tree-based methods
    if feature in top_tree:
        rank = top_tree.index(feature) + 1
        votes += (top_k - rank + 1) / top_k
    
    # Vote from RFECV
    if feature in top_rfecv:
        votes += 1.5  # RFECV gets extra weight (validated through CV)
    
    feature_votes[feature] = votes

# Create voting dataframe
voting_df = pd.DataFrame({
    'Feature': list(feature_votes.keys()),
    'Votes': list(feature_votes.values())
}).sort_values('Votes', ascending=False)

# Add method indicators
voting_df['In_Statistical_Top60'] = voting_df['Feature'].isin(top_statistical)
voting_df['In_Tree_Top60'] = voting_df['Feature'].isin(top_tree)
voting_df['In_RFECV'] = voting_df['Feature'].isin(top_rfecv)
voting_df['Method_Count'] = (
    voting_df['In_Statistical_Top60'].astype(int) +
    voting_df['In_Tree_Top60'].astype(int) +
    voting_df['In_RFECV'].astype(int)
)

print(f"\n Voting completed!")
print(f"\n Top 40 Features by Ensemble Voting:")
print(voting_df.head(40))

# Visualize voting results
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# Top features by votes
top_30_votes = voting_df.head(30)
axes[0].barh(range(len(top_30_votes)), top_30_votes['Votes'],
            color='darkgreen', edgecolor='black', alpha=0.7)
axes[0].set_yticks(range(len(top_30_votes)))
axes[0].set_yticklabels([f[:35] + '...' if len(f) > 35 else f 
                        for f in top_30_votes['Feature']], fontsize=9)
axes[0].set_xlabel('Vote Score', fontsize=11)
axes[0].set_title('Top 30 Features - Ensemble Voting', fontweight='bold', fontsize=13)
axes[0].invert_yaxis()
axes[0].grid(alpha=0.3, axis='x')

# Method agreement
method_counts = voting_df['Method_Count'].value_counts().sort_index()
axes[1].bar(method_counts.index, method_counts.values,
           color=['#d62728', '#ff7f0e', '#2ca02c', '#1f77b4'],
           edgecolor='black', alpha=0.7)
axes[1].set_xlabel('Number of Methods Agreeing', fontsize=11)
axes[1].set_ylabel('Number of Features', fontsize=11)
axes[1].set_title('Feature Selection Method Agreement', fontweight='bold', fontsize=13)
axes[1].set_xticks([0, 1, 2, 3])
axes[1].set_xticklabels(['0 methods', '1 method', '2 methods', '3 methods'])
axes[1].grid(alpha=0.3, axis='y')

# Add text annotations
for i, count in enumerate(method_counts.values):
    axes[1].text(method_counts.index[i], count + 1, str(count),
                ha='center', fontweight='bold', fontsize=11)

plt.tight_layout()
plt.show()

# Select final features (threshold: at least 2 methods agree OR top 50 by votes)
final_features_threshold = voting_df[voting_df['Method_Count'] >= 2]['Feature'].tolist()
final_features_top = voting_df.head(60)['Feature'].tolist()

# Combine both criteria
final_selected_features = list(set(final_features_threshold + final_features_top))

print(f"\n Final Feature Selection:")
print(f"   Features selected by ≥2 methods: {len(final_features_threshold)}")
print(f"   Top 60 by voting: {len(final_features_top)}")
print(f"   Total unique features: {len(final_selected_features)}")

print(f"\n Final Selected Features ({len(final_selected_features)}):")
final_features_sorted = voting_df[voting_df['Feature'].isin(final_selected_features)].copy()
final_features_sorted = final_features_sorted.sort_values('Votes', ascending=False)

for i, row in enumerate(final_features_sorted.head(30).itertuples(), 1):
    methods = []
    if row.In_Statistical_Top60:
        methods.append('Stat')
    if row.In_Tree_Top60:
        methods.append('Tree')
    if row.In_RFECV:
        methods.append('RFECV')
    
    print(f"   {i:2d}. {row.Feature[:50]:<50} | Votes: {row.Votes:.2f} | [{', '.join(methods)}]")

if len(final_features_sorted) > 30:
    print(f"   ... and {len(final_features_sorted) - 30} more features")

# Save final feature list
final_features_list = final_features_sorted['Feature'].tolist()

In [ ]:
# ============================================
# STEP 23: ANALYZE FINAL FEATURE SET
# ============================================

print("\n" + "="*60)
print("ANALYZING FINAL FEATURE SET")
print("="*60)

# Categorize features by type
feature_categories = {
    'Time Series': [],
    'Temporal': [],
    'Weather': [],
    'Pollutants': [],
    'Location Aggregations': [],
    'Wind Features': [],
    'Interactions': [],
    'Ratios': [],
    'Polynomial': [],
    'PCA': [],
    'Clustering': [],
    'Other': []
}

for feature in final_features_list:
    if 'rolling' in feature or 'lag' in feature or 'diff' in feature:
        feature_categories['Time Series'].append(feature)
    elif 'month' in feature or 'day' in feature or 'season' in feature or 'weekend' in feature:
        feature_categories['Temporal'].append(feature)
    elif 'temperature' in feature or 'humidity' in feature or 'precipitable' in feature:
        feature_categories['Weather'].append(feature)
    elif 'L3_' in feature and any(x in feature for x in ['NO2', 'CO', 'SO2', 'HCHO', 'O3']):
        feature_categories['Pollutants'].append(feature)
    elif 'place_' in feature:
        feature_categories['Location Aggregations'].append(feature)
    elif 'wind' in feature.lower():
        feature_categories['Wind Features'].append(feature)
    elif 'interaction' in feature or 'pollutant_load' in feature or 'AQI' in feature:
        feature_categories['Interactions'].append(feature)
    elif 'ratio' in feature:
        feature_categories['Ratios'].append(feature)
    elif 'squared' in feature or 'sqrt' in feature or 'log' in feature:
        feature_categories['Polynomial'].append(feature)
    elif 'pca' in feature.lower():
        feature_categories['PCA'].append(feature)
    elif 'cluster' in feature:
        feature_categories['Clustering'].append(feature)
    else:
        feature_categories['Other'].append(feature)

print(f"\n Feature Breakdown by Category:")
print(f"{'='*60}")

total_check = 0
for category, features in feature_categories.items():
    if len(features) > 0:
        print(f"\n{category}: {len(features)} features")
        for feature in features[:5]:
            print(f"   - {feature}")
        if len(features) > 5:
            print(f"   ... and {len(features) - 5} more")
        total_check += len(features)

print(f"\n{'='*60}")
print(f"Total features: {total_check}")

# Visualize category distribution
category_counts = {k: len(v) for k, v in feature_categories.items() if len(v) > 0}
category_counts = dict(sorted(category_counts.items(), key=lambda x: x[1], reverse=True))

plt.figure(figsize=(12, 6))
plt.bar(range(len(category_counts)), list(category_counts.values()),
       color='skyblue', edgecolor='black', alpha=0.8)
plt.xticks(range(len(category_counts)), list(category_counts.keys()), rotation=45, ha='right')
plt.xlabel('Feature Category', fontsize=12)
plt.ylabel('Number of Features', fontsize=12)
plt.title('Final Feature Set - Category Distribution', fontweight='bold', fontsize=14)
plt.grid(alpha=0.3, axis='y')

# Add count labels on bars
for i, count in enumerate(category_counts.values()):
    plt.text(i, count + 0.5, str(count), ha='center', fontweight='bold', fontsize=10)

plt.tight_layout()
plt.show()

print(f"\n Feature analysis completed!")

In [ ]:
# ============================================
# STEP 24: PREPARE FINAL DATASET
# ============================================

print("="*60)
print("PREPARING FINAL DATASET FOR MODELING")
print("="*60)

# Create final dataset with selected features
print(f"\n Creating final dataset with {len(final_features_list)} selected features...")

# Features for modeling
X_final = df_fe[final_features_list].copy()
y_final = df_fe['target_log'].copy()  # Using log-transformed target
y_original = df_fe['target'].copy()   # Keep original for reference

# Metadata to keep
metadata_cols = ['Date', 'Place_ID']
metadata = df_fe[metadata_cols].copy()

print(f"\n Final dataset prepared!")
print(f"   Features: {X_final.shape[1]}")
print(f"   Samples: {X_final.shape[0]}")
print(f"   Target: log-transformed PM2.5")

# Check for any remaining missing values
missing_final = X_final.isnull().sum()
if missing_final > 0:
    print(f"\n Warning: {missing_final} missing values remaining")
    print(f"   Filling with median...")
    X_final = X_final.fillna(X_final.median())
    print(f"    Missing values handled")
else:
    print(f"\n No missing values!")

# Train-test split
X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(
    X_final, y_final, test_size=0.2, random_state=42
)

# Also split metadata and original target
metadata_train, metadata_test = train_test_split(
    metadata, test_size=0.2, random_state=42
)

y_train_original, y_test_original = train_test_split(
    y_original, test_size=0.2, random_state=42
)

print(f"\n Train-Test Split:")
print(f"   Training set: {X_train_final.shape}")
print(f"   Test set: {X_test_final.shape}")
print(f"   Split ratio: 80-20")

# Summary statistics
print(f"\n Final Dataset Summary:")
print(f"{'='*60}")
print(f"Total features: {X_final.shape[1]}")
print(f"Total samples: {X_final.shape[0]}")
print(f"Training samples: {X_train_final.shape[0]}")
print(f"Test samples: {X_test_final.shape[0]}")
print(f"\nTarget variable (log-transformed):")
print(f"   Mean: {y_final.mean():.3f}")
print(f"   Std: {y_final.std():.3f}")
print(f"   Min: {y_final.min():.3f}")
print(f"   Max: {y_final.max():.3f}")
print(f"   Skewness: {y_final.skew():.3f}")

# Visualize final feature importance across all selected features
print(f"\n📊 Generating final feature importance visualization...")

# Get importance scores for final features
final_feature_scores = voting_df[voting_df['Feature'].isin(final_features_list)].copy()
final_feature_scores = final_feature_scores.sort_values('Votes', ascending=False).head(40)

plt.figure(figsize=(14, 10))
plt.barh(range(len(final_feature_scores)), final_feature_scores['Votes'],
        color='mediumseagreen', edgecolor='black', alpha=0.8)
plt.yticks(range(len(final_feature_scores)),
          [f[:40] + '...' if len(f) > 40 else f for f in final_feature_scores['Feature']],
          fontsize=9)
plt.xlabel('Vote Score', fontsize=12)
plt.title('Top 40 Features in Final Selection - Vote Scores', fontweight='bold', fontsize=14)
plt.gca().invert_yaxis()
plt.grid(alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

In [ ]:
# ============================================
# STEP 25: SAVE PROCESSED DATA AND ARTIFACTS
# ============================================

print("="*60)
print("SAVING PROCESSED DATA AND ARTIFACTS")
print("="*60)

import pickle
import os

# Create directories if they don't exist
os.makedirs('../data/processed', exist_ok=True)
os.makedirs('../artifacts', exist_ok=True)
os.makedirs('../artifacts/feature_selection', exist_ok=True)

print(f"\n Saving datasets...")

# 1. Save train and test sets
X_train_final.to_csv('../data/processed/X_train.csv', index=False)
X_test_final.to_csv('../data/processed/X_test.csv', index=False)

y_train_final.to_csv('../data/processed/y_train.csv', index=False)
y_test_final.to_csv('../data/processed/y_test.csv', index=False)

# Save original targets too
y_train_original.to_csv('../data/processed/y_train_original.csv', index=False)
y_test_original.to_csv('../data/processed/y_test_original.csv', index=False)

# Save metadata
metadata_train.to_csv('../data/processed/metadata_train.csv', index=False)
metadata_test.to_csv('../data/processed/metadata_test.csv', index=False)

print(f"    Saved train/test datasets")

# 2. Save feature lists
with open('../artifacts/feature_selection/final_features.txt', 'w') as f:
    for feature in final_features_list:
        f.write(f"{feature}\n")

print(f"    Saved final feature list ({len(final_features_list)} features)")

# 3. Save feature scores
voting_df.to_csv('../artifacts/feature_selection/feature_voting_scores.csv', index=False)
all_features_scores.to_csv('../artifacts/feature_selection/all_feature_scores.csv', index=False)
tree_importance_combined.to_csv('../artifacts/feature_selection/tree_importance.csv', index=False)
combined_scores.to_csv('../artifacts/feature_selection/statistical_scores.csv', index=False)

print(f"    Saved feature importance scores")

# 4. Save feature categories
with open('../artifacts/feature_selection/feature_categories.txt', 'w') as f:
    for category, features in feature_categories.items():
        if len(features) > 0:
            f.write(f"\n{category} ({len(features)} features):\n")
            for feature in features:
                f.write(f"  - {feature}\n")

print(f"    Saved feature categories")

# 5. Save complete engineered dataset (before feature selection)
df_fe.to_csv('../data/processed/data_fully_engineered.csv', index=False)

print(f"    Saved complete engineered dataset")

# 6. Create and save feature engineering summary
summary = {
    'original_features': len(df.columns),
    'engineered_features': len(df_fe.columns),
    'final_selected_features': len(final_features_list),
    'train_samples': len(X_train_final),
    'test_samples': len(X_test_final),
    'target_transformation': 'log1p',
    'missing_value_strategy': 'KNN + Group Imputation',
    'outlier_handling': 'IQR Capping',
    'feature_selection_methods': [
        'Variance Threshold',
        'Correlation Filter',
        'SelectKBest (F-stat + MI)',
        'Tree-Based Importance (RF + GB + XGB)',
        'RFECV',
        'Ensemble Voting'
    ]
}

import json
with open('../artifacts/feature_selection/feature_engineering_summary.json', 'w') as f:
    json.dump(summary, f, indent=4)

print(f"    Saved feature engineering summary")

print(f"\n{'='*60}")
print(f" ALL DATA AND ARTIFACTS SAVED SUCCESSFULLY!")
print(f"{'='*60}")

print(f"\n Saved Files:")
print(f"   ../data/processed/")
print(f"      - X_train.csv ({X_train_final.shape})")
print(f"      - X_test.csv ({X_test_final.shape})")
print(f"      - y_train.csv")
print(f"      - y_test.csv")
print(f"      - y_train_original.csv")
print(f"      - y_test_original.csv")
print(f"      - metadata_train.csv")
print(f"      - metadata_test.csv")
print(f"      - data_fully_engineered.csv ({df_fe.shape})")
print(f"\n   ../artifacts/feature_selection/")
print(f"      - final_features.txt")
print(f"      - feature_voting_scores.csv")
print(f"      - all_feature_scores.csv")
print(f"      - tree_importance.csv")
print(f"      - statistical_scores.csv")
print(f"      - feature_categories.txt")
print(f"      - feature_engineering_summary.json")